In [ ]:
import sys
import math
import random
from queue import PriorityQueue
from collections import deque
from dataclasses import dataclass, field

def log(*args):
    print(*args, file=sys.stderr, flush=True)

CMD = {'^' : 'C', '>' : 'A', 'v' : 'D', '<' : 'E', '.' : 'B'}
W, H, _ = int(input()), int(input()), int(input())

grid = {(x, y): True for y in range(H) for x in range(W)}
seeker, hiders = (0, 0), [(0, 0) for _ in range(4)]
dist =    lambda a,b: abs(a[0]-b[0])+abs(a[1]-b[1])
add_pos = lambda a,b:    (a[0]+b[0],     a[1]+b[1])
dirs = {'<':(-1,0),'>':(1,0),'^':(0,-1),'v':(0,1)}
in_dirs = ['^', '>', 'v', '<']

@dataclass(order=True)
class Step:
    cost: int
    pos: tuple = field(compare=False)
    steps: str = field(compare=False)

def a_star():
    global grid, seeker, hiders

    h = min(hiders, key = lambda h: dist(seeker, h))
    q = PriorityQueue(); q.put(Step(0, seeker, ''))
    visited = {seeker}

    while not q.empty():
        crt = q.get()
        for ds in dirs:
            nxt_pos = add_pos(crt.pos, dirs[ds])
            if nxt_pos in visited or nxt_pos not in grid or not grid[nxt_pos]: continue

            nxt_steps = crt.steps + ds
            if nxt_pos in hiders: return nxt_steps[0]
            
            q.put(Step(len(nxt_steps) + 2*dist(nxt_pos, h), nxt_pos, nxt_steps))
            visited.add(nxt_pos)

def bfs():
    global grid, seeker, hiders

    q = deque([(seeker, '')])
    visited = {seeker}

    best_dir = ('.', W*H)
    while q:
        crt = q.popleft()
        for ds in dirs:
            nxt_pos = add_pos(crt[0], dirs[ds])
            if nxt_pos in visited or nxt_pos not in grid or not grid[nxt_pos]: continue

            nxt_steps = crt[1] + ds
            if nxt_pos in hiders and len(nxt_steps) < best_dir[1]:
                best_dir = nxt_steps[0], len(nxt_steps)
            
            q.append((nxt_pos, nxt_steps))
            visited.add(nxt_pos)
    return best_dir[0]

turn = 0
while True:
    available_directions = in_dirs

    for d in in_dirs:
        c = input()
        if turn > 0 and c == '#':
            grid[add_pos(seeker, dirs[d])] = False

    for i in range(5):
        pos = tuple(map(int, input().split()))
        
        if i < 4: hiders[i] = pos
        else: seeker = pos

    log(f'hiders at {hiders}')
    log(f'seeker at {seeker}')

    # Write an action using print
    if turn == 0:
        print(CMD['.'])
    else:
        # log('astar', a_star())
        print(CMD[bfs() or '.'])
        # print(CMD[random.choice(available_directions)])
    # print('A')
    turn += 1
